In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import tensorflow as tf

# pylint: disable=unused-import
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
# pylint: enable=unused-import

FLAGS = None


def load_graph(filename):
  """Unpersists graph from file as default graph."""
  with tf.gfile.FastGFile(filename, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')


def load_labels(filename):
  """Read in labels, one label per line."""
  return [line.rstrip() for line in tf.gfile.GFile(filename)]


def run_graph(wav_data, labels, input_layer_name, output_layer_name,
              num_top_predictions):
  """Runs the audio data through the graph and prints predictions."""
  with tf.Session() as sess:
    # Feed the audio data as input to the graph.
    #   predictions  will contain a two-dimensional array, where one
    #   dimension represents the input image count, and the other has
    #   predictions per class
    softmax_tensor = sess.graph.get_tensor_by_name(output_layer_name)
    #print(wav_data)
    predictions, = sess.run(softmax_tensor, {input_layer_name: wav_data})
    #print(len(predictions))
    # Sort to show labels in order of confidence
    top_k = predictions.argsort()[-num_top_predictions:][::-1]
    for node_id in top_k:
      human_string = labels[node_id]
      score = predictions[node_id]
      
      print('%s (score = %.5f)' % (human_string, score))

    return 0


def label_wav(wav, labels, graph, input_name, output_name, how_many_labels):
  """Loads the model and labels, and runs the inference to print predictions."""
  if not wav or not tf.gfile.Exists(wav):
    tf.logging.fatal('Audio file does not exist %s', wav)

  if not labels or not tf.gfile.Exists(labels):
    tf.logging.fatal('Labels file does not exist %s', labels)

  if not graph or not tf.gfile.Exists(graph):
    tf.logging.fatal('Graph file does not exist %s', graph)

  labels_list = load_labels(labels)

  # load graph, which is stored in the default session
  load_graph(graph)

  with open(wav, 'rb') as wav_file:
    wav_data = wav_file.read()
  #print(wav_data)
  #print(len(wav_data))
  #print(type(wav_data))

  #print(labels_list)
  #print(input_name, output_name, how_many_labels)
  run_graph(wav_data, labels_list, input_name, output_name, how_many_labels)




C:\Users\bhogal\.conda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\bhogal\.conda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\bhogal\.conda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\bhogal\.conda\envs\keras\lib\site-packages\tensorflow\python\framework

In [ ]:
label_wav('silence.wav','Pretrained_models\labels.txt','DNN_S.pb','wav_data:0','labels_softmax:0',5)

In [3]:
def label_wav_data(wav, labels, graph, input_name, output_name, how_many_labels):
  """Loads the model and labels, and runs the inference to print predictions."""
  if not wav or not tf.gfile.Exists(wav):
    tf.logging.fatal('Audio file does not exist %s', wav)

  if not labels or not tf.gfile.Exists(labels):
    tf.logging.fatal('Labels file does not exist %s', labels)

  if not graph or not tf.gfile.Exists(graph):
    tf.logging.fatal('Graph file does not exist %s', graph)

  labels_list = load_labels(labels)

  # load graph, which is stored in the default session
  load_graph(graph)



  print(labels_list)
  print(input_name, output_name, how_many_labels)
  run_graph(wav, labels_list, input_name, output_name, how_many_labels)

In [4]:
import pyaudio
import wave
from array import array

FORMAT=pyaudio.paInt16
CHANNELS=1
RATE=16000
CHUNK=1024
RECORD_SECONDS=1
FILE_NAME="RECORDING.wav"

audio=pyaudio.PyAudio() #instantiate the pyaudio

#recording prerequisites
stream=audio.open(format=FORMAT,channels=CHANNELS, 
                  rate=RATE,
                  input=True,
                  frames_per_buffer=CHUNK)

#starting recording
frames=[]
while(True):
    data=stream.read(CHUNK)
    data_chunk=array('h',data)
    vol=max(data_chunk)
    print('.')
    if(vol>=1000):
        print('Triggered')
        break
for i in range(0,int(RATE/CHUNK*RECORD_SECONDS)):
    data=stream.read(CHUNK)
    data_chunk=array('h',data)
    vol=max(data_chunk)
    #if(vol>=300):
     #   print("something said")
    frames.append(data)
    #else:
        #print("nothing")
    #print("\n")


#end of recording
stream.stop_stream()
stream.close()
audio.terminate()
#writing to file
wavfile=wave.open(FILE_NAME,'wb')
wavfile.setnchannels(CHANNELS)
wavfile.setsampwidth(audio.get_sample_size(FORMAT))
wavfile.setframerate(RATE)
wavfile.writeframes(b''.join(frames))#append frames recorded to file
wavfile.close()

label_wav('RECORDING.wav','Pretrained_models\labels.txt','DNN_S.pb','wav_data:0','labels_softmax:0',1)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
Triggered
Instructions for updating:
Use tf.gfile.GFile.
left (score = 0.34399)


In [24]:
import pyaudio
import wave
from array import array



FORMAT=pyaudio.paInt16
CHANNELS=1
RATE=16000
CHUNK=1024
CHUNK_THRESH=64
RECORD_SECONDS=1
FILE_NAME="RECORDING.wav"
while(True):
    audio=pyaudio.PyAudio() #instantiate the pyaudio

    #recording prerequisites
    stream=audio.open(format=FORMAT,channels=CHANNELS, 
                      rate=RATE,
                      input=True,
                      frames_per_buffer=CHUNK)

    #starting recording
    frames=[]
    while(True):
        old_data=stream.read(CHUNK_THRESH)
        data_chunk=array('h',old_data)
        vol=max(data_chunk)
        #print(old_data)
        if(vol>=600):
            print('Triggered')
            #frames.append(old_data)
            break
    for i in range(0,int(RATE/CHUNK*RECORD_SECONDS)):
        #frames.append(old_data)
        data=stream.read(CHUNK)
        data_chunk=array('h',data)
        vol=max(data_chunk)
        #if(vol>=300):
         #   print("something said")
        frames.append(data)
        #else:
            #print("nothing")
        #print("\n")


    #end of recording
    stream.stop_stream()
    stream.close()
    audio.terminate()
    #writing to file
    wavfile=wave.open(FILE_NAME,'wb')
    wavfile.setnchannels(CHANNELS)
    wavfile.setsampwidth(2)
    wavfile.setframerate(RATE)
    wavfile.writeframes(b''.join(frames))#append frames recorded to file
    wavfile.close()

    label_wav('RECORDING.wav','Pretrained_models\labels.txt','DNN_S.pb','wav_data:0','labels_softmax:0',1)

Triggered
on (score = 0.22909)
Triggered
left (score = 0.63180)
Triggered
on (score = 0.76567)
Triggered
on (score = 0.64561)
Triggered
right (score = 0.92620)
Triggered
down (score = 0.41801)
Triggered
left (score = 0.85192)
Triggered
yes (score = 0.39493)
Triggered
left (score = 0.24423)
Triggered
right (score = 0.53452)
Triggered
right (score = 0.60255)


KeyboardInterrupt: 

In [25]:
label_wav('RECORDING.wav','Pretrained_models\labels.txt','DNN_S.pb','wav_data:0','labels_softmax:0',1)

right (score = 0.60255)


In [16]:
import os
import wave
with wave.open('silence.wav', "rb") as wave_file:
        frame_rate = wave_file.getframerate()

In [17]:
frame_rate

16000

In [18]:
wave_file = wave.open('silence.wav', "rb")

In [23]:
wave_file.getfp()

In [ ]:
str(frames[0])

In [ ]:
label_wav_data(frames[0],'Pretrained_models\labels.txt','DNN_S.pb','wav_data:0','labels_softmax:0',1)

In [ ]:
tf.audio.decode_wav(frames[0])

In [ ]:
'RIFF$}\x00\x00WAVEfmt '+str(frames[0])

In [ ]:
import pyaudio
import wave
from array import array

FORMAT=pyaudio.paInt16
CHANNELS=1
RATE=16000
CHUNK=1024
RECORD_SECONDS=2
FILE_NAME="RECORDING.wav"

audio=pyaudio.PyAudio() #instantiate the pyaudio

#recording prerequisites
stream=audio.open(format=FORMAT,channels=CHANNELS, 
                  rate=RATE,
                  input=True,
                  frames_per_buffer=CHUNK)

#starting recording
frames=[]

for i in range(0,int(RATE/CHUNK*RECORD_SECONDS)):
    data=stream.read(CHUNK)
    
    data_chunk=array('h',data)
    vol=max(data_chunk)
    #if(vol>=300):
     #   print("something said")
    frames.append(data)
    #else:
        #print("nothing")
    #print("\n")


#end of recording
stream.stop_stream()
stream.close()
audio.terminate()
#writing to file
print(frames)

In [ ]:
print("Wait in silence to begin recording; wait in silence to terminate")
record_to_file('demo.wav')
print("done - result written to demo.wav")


In [ ]:
import pyaudio
import sys

chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 1

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS, 
                rate=RATE, 
                input=True,
                output=True,
                frames_per_buffer=chunk)

print ("* recording")
for i in range(0, int(44100 / chunk * RECORD_SECONDS)):
    data = stream.read(chunk)
    # check for silence here by comparing the level with 0 (or some threshold) for 
    # the contents of data.
    # then write data or not to a file

print ("* done")

stream.stop_stream()
stream.close()
p.terminate()
with open()

In [ ]:
data

In [ ]:
label_wav(data,'Pretrained_models\labels.txt','DNN_L.pb','wav_data:0','labels_softmax:0',5)

In [ ]:
labels_list=['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
input_name='wav_data:0' 
output_name='labels_softmax:0'
how_many_labels=5


In [ ]:
record_to_file('demo.wav')
print("done - result written to demo.wav")

In [ ]:
with open('silence.wav', 'rb') as wav_file:
    wav_data = wav_file.read()

In [ ]:
wav_data

In [70]:
RATE

16000

In [82]:
SILENCE_LIMIT = 1  # Silence limit in seconds. The max ammount of seconds where
                   # only silence is recorded. When this time passes the
                   # recording finishes and the file is delivered.

PREV_AUDIO = 0.2  # Previous audio (in seconds) to prepend. When noise
                  # is detected, how much of previously recorded audio is
                  # prepended. This helps to prevent chopping the beggining
                  # of the phrase.
import math
import audioop
import time
from collections import deque
def listen_for_speech(threshold=400, num_phrases=30):
    THRESHOLD=400
    """
    Listens to Microphone, extracts phrases from it and sends it to 
    Google's TTS service and returns response. a "phrase" is sound 
    surrounded by silence (according to threshold). num_phrases controls
    how many phrases to process before finishing the listening process 
    (-1 for infinite). 
    """

    #Open stream
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print ("* Listening mic. ")
    audio2send = []
    cur_data = ''  # current chunk  of audio data
    rel = RATE/CHUNK
    slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
    #Prepend audio from 0.5 seconds before noise was detected
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel)) 
    started = False
    n = num_phrases
    response = []

    while (num_phrases == -1 or n > 0):
        cur_data = stream.read(CHUNK)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        #print slid_win[-1]
        if(sum([x > THRESHOLD for x in slid_win]) > 0):
            if(not started):
                print ("Starting record of phrase")
                started = True
            audio2send.append(cur_data)
        elif (started is True):
            print ("Finished")
            # The limit was reached, finish capture and deliver.
            filename = save_speech(list(prev_audio) + audio2send, p)
            label_wav(filename,'Pretrained_models\labels.txt','DNN_S.pb','wav_data:0','labels_softmax:0',1)

            # Send file to Google and get response
            #r = stt_google_wav(filename) 
            #if num_phrases == -1:
                #print ("Response", r)
            #else:
                #response.append(r)
            # Remove temp file. Comment line to review.
            os.remove(filename)
            # Reset all
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=int(0.5 * rel)) 
            audio2send = []
            n -= 1
            print ("Listening ...")
        else:
            prev_audio.append(cur_data)

    print ("* Done recording")
    stream.close()
    p.terminate()

    return response


def save_speech(data, p):
    """ Saves mic data to temporary WAV file. Returns filename of saved 
        file """

    filename = 'output_'+str(int(time.time()))
    # writes data to WAV file
    #data = ''.join(data)
    
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(16000)  # TODO make this value a function parameter?
    #wf.writeframes(data)
    wf.writeframes(b''.join(data))
    wf.close()
    return filename + '.wav'




In [83]:
listen_for_speech()

* Listening mic. 
Starting record of phrase
Finished
on (score = 0.70106)
Listening ...
Starting record of phrase
Finished
off (score = 0.99679)
Listening ...
Starting record of phrase
Finished
right (score = 0.98753)
Listening ...
Starting record of phrase
Finished
left (score = 0.52431)
Listening ...
Starting record of phrase
Finished
right (score = 0.99920)
Listening ...
Starting record of phrase
Finished
left (score = 0.97932)
Listening ...
Starting record of phrase
Finished
off (score = 0.99714)
Listening ...
Starting record of phrase
Finished


KeyboardInterrupt: 

In [ ]:
def main(_):
  """Entry point for script, converts flags to arguments."""
  label_wav(FLAGS.wav, FLAGS.labels, FLAGS.graph, FLAGS.input_name,
            FLAGS.output_name, FLAGS.how_many_labels)


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--wav', type=str, default='', help='Audio file to be identified.')
  parser.add_argument(
      '--graph', type=str, default='', help='Model to use for identification.')
  parser.add_argument(
      '--labels', type=str, default='', help='Path to file containing labels.')
  parser.add_argument(
      '--input_name',
      type=str,
      default='wav_data:0',
      help='Name of WAVE data input node in model.')
  parser.add_argument(
      '--output_name',
      type=str,
      default='labels_softmax:0',
      help='Name of node outputting a prediction in the model.')
  parser.add_argument(
      '--how_many_labels',
      type=int,
      default=3,
      help='Number of results to show.')

  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)